In [45]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [46]:
# Data Load: Load banglore home prices into a dataframe

In [47]:
df1 = pd.read_csv("resources/bengaluru_house_prices.csv")
df1.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [48]:
# get number of row and column

In [49]:
df1.shape


(13320, 9)

In [50]:
# get all columns
df1.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')

In [51]:
# drop column which is not required for prediction. We have dropped this column because a in generic model area_type and society
# society balcony and avaiability info might not always be present. also textual name does not add much significance
df2 = df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.shape

(13320, 5)

In [52]:
# find nuber of null value in a column
df2.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [53]:
# we can drop this column as we have bigger size of data so deleting some rows won't impact much 
df3 = df2.dropna()
df3.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [54]:
# get shape after dropping 
df3.shape

(13246, 5)

In [55]:
# let start data formatting here we can see some of room mentioned as 
# bedroom and some as bhk so we will bring it down to smae fromat.
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
df3.bhk.unique()

<ipython-input-55-5f638e55aad6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))


array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [56]:
df3.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [57]:
df3.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [58]:
# from above we can see some area has sqft in range. To handle them we will use mean of them 
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [59]:
df4 = df3.copy()
df4.total_sqft = df4.total_sqft.apply(convert_sqft_to_num)
df4 = df4[df4.total_sqft.notnull()]
df4.head(2)

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4


In [60]:
# check now if total_sqft is smae format
df4.total_sqft.unique

<bound method Series.unique of 0        1056.0
1        2600.0
2        1440.0
3        1521.0
4        1200.0
          ...  
13315    3453.0
13316    3600.0
13317    1141.0
13318    4689.0
13319     550.0
Name: total_sqft, Length: 13200, dtype: float64>

In [61]:
# feature engineering
# we are going to create new column price_per_sqft. Here price is in lakh so converting to rupees
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [62]:
# Examine locations which is a categorical variable. We need to apply dimensionality 
# reduction technique here to reduce number of locations. 
# Hot encoding or dummy variable don't work well with categorical varibale with too many unique value
len(df5.location.unique())

1298

In [63]:
# let count number of property per locality. We are removing extra space before 
# and after location name just for precaution that two name should not be shown different just because of spaces.
df5.location = df5.location.apply(lambda x: x.strip())
location_stats = df5['location'].value_counts(ascending=False)
location_stats

Whitefield                                533
Sarjapur  Road                            392
Electronic City                           304
Kanakpura Road                            264
Thanisandra                               235
                                         ... 
Kudlu Village,                              1
anjananager magdi road                      1
Satyasaibaba Layout                         1
Banashankari 6th Stage ,Subramanyapura      1
Meenakshi Layout                            1
Name: location, Length: 1287, dtype: int64

In [64]:
location_stats.values.sum()

13200

In [65]:
len(location_stats[location_stats>10])

240

In [66]:
# Dimensionality Reduction
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

Thyagaraja Nagar                          10
Nagadevanahalli                           10
Nagappa Reddy Layout                      10
Basapura                                  10
Gunjur Palya                              10
                                          ..
Kudlu Village,                             1
anjananager magdi road                     1
Satyasaibaba Layout                        1
Banashankari 6th Stage ,Subramanyapura     1
Meenakshi Layout                           1
Name: location, Length: 1047, dtype: int64

In [67]:
# total unique location before dimension reduction
len(df5.location.unique())

1287

In [68]:
# Any location having less than 10 data points should be tagged as "other" location. 
# This way number of categories can be reduced by huge amount. Later on when we do one hot encoding, 
# it will help us with having fewer dummy columns

# total unique location after dimension reduction
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

241

In [69]:
df5.head(10)

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,3247.863248
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,7467.057101
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,18181.818182
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,4828.244275
9,other,6 Bedroom,1020.0,6.0,370.00,6,36274.509804


In [70]:
# Outlier Removal Using Business Logic
# As a data scientist when you have a conversation with your business manager 
# (who has expertise in real estate), he will tell you that normally square ft 
# per bedroom is 300 (i.e. 2 bhk apartment is minimum 600 sqft. If you have for 
# example 400 sqft apartment with 2 bhk than that seems suspicious and can be removed 
# as an outlier. We will remove such outliers by keeping our minimum thresold per bhk to be 300 sqft

# Check below data points. We have 6 bhk apartment with 1020 sqft. Another one is 8 bhk and total sqft is 600. 
# These are clear data errors that can be removed safely

df5[df5.total_sqft/df5.bhk<300].head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
9,other,6 Bedroom,1020.0,6.0,370.0,6,36274.509804
45,HSR Layout,8 Bedroom,600.0,9.0,200.0,8,33333.333333
58,Murugeshpalya,6 Bedroom,1407.0,4.0,150.0,6,10660.980810
68,Devarachikkanahalli,8 Bedroom,1350.0,7.0,85.0,8,6296.296296
70,other,3 Bedroom,500.0,3.0,100.0,3,20000.000000


In [71]:
# discard property whose area is less than 300
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

(12456, 7)

In [72]:
# Outlier Removal Using Standard Deviation and Mean
df6.price_per_sqft.describe()

count     12456.000000
mean       6308.502826
std        4168.127339
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64

In [73]:
# Here we find that min price per sqft is 267 rs/sqft whereas max is 12000000, this shows a wide 
# variation in property prices. We should remove outliers per location 
# using mean and one standard deviation

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft) # mean
        st = np.std(subdf.price_per_sqft) # standard deviation
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))] # removing outlier per locality 
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7 = remove_pps_outliers(df6)
df7.shape